# Nexus-hosted co-occurrence network analysis of CORD-19

In this notebook we will illustrate how different datasets from the co-occurrence analysis can be pushed and accessed within a Blue Brain Nexus project. To get

In [ ]:
import getpass
import jwt
import nexussdk as nexus
import yaml

from kgforge.core import KnowledgeGraphForge

from cord19kg.apps.topic_widgets import (TopicWidget, DataSaverWidget)

## 1. Setting up a project and creating a `kgforge` configuration

If you have already set up a Nexus project and generated a 'forge' configuration, simply get your access token (__1.1__) and go directly to the step __2. Set up a topic__.

### 1.1. Login to Nexus and get the access token

The [Nexus web application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token:

1. Click on the login button on the right corner and follow the instructions.
<img src="../figures/nexus_log_in.png" alt="Drawing" style="width: 1000px;"/>


2. Once logged in, click on the `Copy token` button. The token will be copied to the clipboard.
<img src="../figures/nexus_logged_in.png" alt="Drawing" style="width: 1000px;"/>


Run the cell below and paste the token in the input field generated by the cell.

In [ ]:
TOKEN = getpass.getpass()

### 1.2. Create a Nexus project programmatically

In the cell below modify the variable `project` to chose a new project name.

In [ ]:
org ="tutorialnexus"
project ="cord19kgExampleProject"  # Choose a project name
description = "cord19kg save/load example project"
endpoint = "https://sandbox.bluebrainnexus.io/v1"

nexus.config.set_environment(endpoint)
nexus.config.set_token(TOKEN)

# nexus.projects.create(org_label=org,project_label=project, description=description)

### 1.3. Generate a `kgforge` configuration for your project

The following cell automatically generates a configuration file for the project set up above.

In [ ]:
config = dict()

neuroshapes_path = "../models/neuroshapes"
! rm -Rf $neuroshapes_path
! git clone https://github.com/INCF/neuroshapes.git $neuroshapes_path
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/dataset $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/activity $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/entity $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/ontology $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/person $neuroshapes_path/shapes/neurosciencegraph/commons/

config['Model'] = {
    "name": "RdfModel",
    "origin": "directory",
    "source": f"{neuroshapes_path}/shapes/neurosciencegraph/commons/",
    "context": {
        "iri": "../models/neuroshapes_context.json",
    },
}

config["Store"] = {
    "name": "BlueBrainNexus",
    "endpoint": endpoint,
    "searchendpoints":{
        "sparql":{
            "endpoint":"https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex"
        }
    },
    "bucket": f"{org}/{project}",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../config/file-to-resource-mapping.hjson"
}

with open("../config/forge-config.yml", "w") as f:
    yaml.dump(config, f)

## 2. Set up a topic

Create a 'forge' to manage (create, access and deploy) knowledge within the given Blue Brain Nexus Project.

In [ ]:
forge_config_file = "../config/forge-config.yml"
forge = KnowledgeGraphForge(forge_config_file, token=TOKEN, debug=True)

In [ ]:
agent_username = jwt.decode(TOKEN, verify=False)['preferred_username']

In [ ]:
agent_username

In [ ]:
widget = TopicWidget(forge, agent_username)
widget.display()